<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.


In [18]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

In [19]:
eps_5year_over20pct = True
eps_qoq_over20pct = True
eps_yoy_over20pct = True
sales_5years_over20pct = True
sales_qoq_over20pct = True
curr_vol_over200 = True

filters = []
if eps_5year_over20pct:
  filters.append('fa_eps5years_o20')
if eps_qoq_over20pct:
  filters.append('fa_epsqoq_o20')
if eps_yoy_over20pct:
  filters.append('fa_epsyoy_o20')
if sales_5years_over20pct:
  filters.append('fa_sales5years_o20')
if sales_qoq_over20pct:
  filters.append('fa_salesqoq_o20')
if curr_vol_over200:
  filters.append('sh_curvol_o200')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [20]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price   | Change | Volume    
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------- | ------ | ----------
1   | VTVT   | 17.00%    | 7.04%      | -6.77%     | 52.11%    | 57.49%    | 55.38%   | 7.84%        | 10.69%       | 2.00  | 3.43M      | 0.48       | 2.93    | 1.38%  | 1,353,465 
2   | OPK    | 2.65%     | -17.29%    | 4.27%      | 46.37%    | 172.94%   | 17.47%   | 5.90%        | 6.91%        | 1.80  | 8.40M      | 0.44       | 4.78    | 3.12%  | 3,014,427 
3   | CTMX   | 3.41%     | -5.67%     | 12.01%     | 18.25%    | 135.18%   | 29.62%   | 4.61%        | 6.52%        | 2.30  | 951.09K    | 0.36       | 8.54    | 0.59%  | 283,060   
4   | RBBN   | 7.12%     | -18.69%    | 38.90%     | 101.65%   | 296.76%   | 30.64%   | 3.

In [23]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
    screened_stocks[stock['Ticker']] = {}
    
    finviz_stats = finviz.get_stock(stock['Ticker'])
    try:
        yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
    except:
        continue
    SMA200_value = moving_average(yahoo_df, days=200)
    SMA150_value = moving_average(yahoo_df, days=150)
    SMA50_value = moving_average(yahoo_df, days=50)
    SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
    SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
    try:
      inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
    except:
      inst_own = 0
    
    shares_outstanding = finviz_stats['Shs Outstand']
    if 'M' in shares_outstanding:
      shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
    elif 'B' in shares_outstanding:
      shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
    elif 'T' in shares_outstanding:
      shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12

    prev_close = float(finviz_stats['Prev Close'].replace("$",""))
    week52_high, week52_low = week52_low_high(yahoo_df)
    
    screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
    screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
    screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
    screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
    screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
    screened_stocks[stock['Ticker']]['prev_close'] = prev_close
    screened_stocks[stock['Ticker']]['week52_high'] = week52_high
    screened_stocks[stock['Ticker']]['week52_low'] = week52_low
    screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
    screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
    
    # Shares Outstanding <= 25 mil
    if shares_outstanding <= 25e6:
      shares_outstanding_rule = True
    else:
      shares_outstanding_rule = False
    screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

    # Institutional Ownership < 35%
    if inst_own <= 0.35:
      inst_ownership_rule = True
    else:
      inst_ownership_rule = False
    screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

    # Positive 200d MA positive
    if SMA200_percent > 0:
        SMA200_slope_rule = True
    else:
        SMA200_slope_rule = False
    screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
        
    # 150d MA greater than 200d MA
    if SMA150_value > SMA200_value:
        SMA150_greater_SMA200_rule = True
    else:
        SMA150_greater_SMA200_rule = False
    screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
        
    # 50d MA greater than 150d MA
    if SMA50_value > SMA150_value:
        SMA50_greater_SMA150_rule = True
    else:
        SMA50_greater_SMA150_rule = False
    screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
        
    # Close above 50d MA
    if prev_close > SMA50_value:
        close_greater_SMA50_rule = True
    else:
        close_greater_SMA50_rule = False
    screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
        
    # 52 week high low span rule
    if 0.75*week52_high > 1.25*week52_low:
        week52_span_rule = True
    else:
        week52_span_rule = False
    screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
    
    # Close above 52 week high - 25%
    if prev_close > 0.75*week52_high:
        close_above_52weekhigh_rule = True
    else:
        close_above_52weekhigh_rule = False
    screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    




  0%|          | 0/38 [00:00<?, ?it/s]

  3%|▎         | 1/38 [00:01<00:57,  1.55s/it]

  5%|▌         | 2/38 [00:02<00:54,  1.50s/it]

  8%|▊         | 3/38 [00:04<00:52,  1.50s/it]

 11%|█         | 4/38 [00:05<00:51,  1.51s/it]

 13%|█▎        | 5/38 [00:07<00:49,  1.51s/it]

 16%|█▌        | 6/38 [00:09<00:49,  1.55s/it]

 18%|█▊        | 7/38 [00:10<00:49,  1.59s/it]

 21%|██        | 8/38 [00:12<00:48,  1.60s/it]

 24%|██▎       | 9/38 [00:13<00:45,  1.56s/it]

 26%|██▋       | 10/38 [00:15<00:43,  1.57s/it]

 29%|██▉       | 11/38 [00:17<00:42,  1.57s/it]

 32%|███▏      | 12/38 [00:18<00:39,  1.51s/it]

 34%|███▍      | 13/38 [00:20<00:40,  1.60s/it]

 37%|███▋      | 14/38 [00:21<00:38,  1.62s/it]

 39%|███▉      | 15/38 [00:23<00:37,  1.64s/it]

 42%|████▏     | 16/38 [00:25<00:36,  1.66s/it]

 45%|████▍     | 17/38 [00:26<00:33,  1.61s/it]

 47%|████▋     | 18/38 [00:28<00:32,  1.63s/it]

 50%|█████     | 19/38 [00:29<00:29,  1.57s/it]

 53%|█████▎    | 20/38 [00:31<00:28,

In [26]:
df_out.all(axis=1)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32     True
33    False
34    False
35    False
36    False
37    False
dtype: bool

In [39]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [42]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
0,VTVT,2.091022,2.173495,2.528333,29.04,19.05,2.89,3.190000,1.490000,0.0440,5.116000e+07,False,True,True,True,True,True,True,True,7.0
24,NXST,108.575328,113.401650,134.544444,55.80,24.51,158.60,158.600006,46.259998,0.0000,4.374000e+07,False,True,True,True,True,True,True,True,7.0
6,PLYM,14.063029,14.470825,15.620695,21.44,9.65,17.08,17.350000,7.900000,0.6780,2.483000e+07,True,False,True,True,True,True,True,True,7.0
21,FND,87.093905,90.973155,96.689028,25.37,1.66,98.45,105.290001,27.040001,0.0000,1.038400e+08,False,True,True,True,True,True,True,True,7.0
13,GRWG,33.714453,39.595971,51.193889,96.88,1.03,51.78,64.580002,2.870000,0.5370,4.788000e+07,False,False,True,True,True,True,True,True,6.0
27,ETSY,167.013722,180.799659,215.721944,40.48,-1.10,215.98,244.580002,31.690001,0.9000,1.260100e+08,False,False,True,True,True,True,True,True,6.0
26,MKSI,140.788614,150.395049,164.346945,37.77,10.23,177.91,188.240005,72.070000,0.9804,5.510000e+07,False,False,True,True,True,True,True,True,6.0
25,FTNT,138.026351,143.632428,166.399168,33.63,14.13,189.52,192.029999,76.519997,0.6990,1.624000e+08,False,False,True,True,True,True,True,True,6.0
1,OPK,4.258467,4.491068,4.978333,12.23,-2.82,4.64,5.990000,1.150000,0.2920,6.407600e+08,False,True,True,True,True,False,True,True,6.0
17,PFSI,59.590109,60.097136,62.641805,20.56,7.03,65.05,69.489998,16.900000,0.5490,7.177000e+07,False,False,True,True,True,True,True,True,6.0
